In [ ]:
import sys, os
import warnings
warnings.filterwarnings("ignore")
import json
import pandas as pd
import matplotlib.pyplot as plt
DIR = os.getcwd()
sys.path.append(f"{DIR}/../")
import PLACER

## Examples of how to perform various prediction tasks using PLACER Python API

In [ ]:
## Initializing PLACER model with default checkpoint
placer = PLACER.PLACER()

## Prediction tests with native ligands and structures

In [ ]:
print("""
################################
# Predicting heme in myoglobin #
#    PDB input from RCSB       #
################################

## This job should fail because the ligand has the same chain letter as the protein
""")

try:
    pdbfile = f"{DIR}/inputs/3rgk.pdb"
    
    pl_inp = PLACER.PLACERinput()
    pl_inp.pdb(pdbfile)
    pl_inp.name(os.path.basename(pdbfile).replace(".pdb", ""))
    pl_inp.ligand_reference({"HEM": "CCD"})
    outputs_Mb = placer.run(pl_inp, 10)
except AssertionError as e:
    print("AssertionError:\n  ", e)
    pass

In [ ]:
print("""
################################
# Predicting heme in myoglobin #
#    CIF input from RCSB       #
################################
""")

## This works becaus CIF parser handles ligand parsing differently
cif_file = f"{DIR}/inputs/3rgk.cif"

pl_inp = PLACER.PLACERinput()
pl_inp.cif(cif_file)
pl_inp.name(os.path.basename(cif_file).replace(".cif", ""))
pl_inp.skip_ligands(["HOH", "SO4"])
pl_inp.ligand_reference({"HEM": "CCD"})
outputs_Mb_cif = placer.run(pl_inp, 10)

In [ ]:
print("""
#####################################
# Predicting dopamine bound to P450 #
# while keeping heme fixed.         #
#    CIF input from RCSB            #
#####################################
""")

cif_file = f"{DIR}/inputs/4dtz.cif"

pl_inp = PLACER.PLACERinput()
pl_inp.cif(cif_file)
pl_inp.name(os.path.basename(cif_file).replace(".cif", ""))
# enabling predict_ligand() automatically fixes all other ligands that are not defined here
# "fixing ligands" means that they are minimally noised, increasing the likelihood that they are predicted similar to their input pose.
pl_inp.predict_ligand([("D", "LDP", 501)])
pl_inp.ligand_reference({"HEM": "CCD", "LDP": "CCD"})
outputs_dopa = placer.run(pl_inp, 10)

## Prediction tests with non-native ligands

In [ ]:
print("""
##############################################
# Predicting a designed heme-binding protein #
# with heme atoms being renamed.             #
#    PDB input generated by Rosetta          #
##############################################
""")

pdbfile = f"{DIR}/inputs/dnHEM1.pdb"

pl_inp = PLACER.PLACERinput()
pl_inp.pdb(pdbfile)
pl_inp.name(os.path.basename(pdbfile).replace(".pdb", ""))
pl_inp.ligand_reference({"HEM": f"{DIR}/ligands/HEM.mol2"})
outputs_denovo = placer.run(pl_inp, 10)

In [ ]:
print("""
##############################################
# Predicting a designed heme-binding protein #
# with a non-standard heme atoms naming.     #
#    PDB input generated by Rosetta          #
#  Using input from a dictionary             #
##############################################
""")

pdbfile = f"{DIR}/inputs/dnHEM1.pdb"
pdbstr = open(pdbfile, "r").read()

ODIR = "example_out_API/outputs_denovo2"

inp_dict = {"ligand_reference": {"HEM": f"{DIR}/ligands/HEM.mol2"},
            "name": os.path.basename(pdbfile).replace(".pdb", ""),
            "pdb": pdbstr}

pl_inp = PLACER.PLACERinput()
pl_inp.create_from_dict(inp_dict)
outputs_denovo2 = placer.run(pl_inp, 50)

# Ranking outputs by prmsd
outputs_denovo2 = PLACER.utils.rank_outputs(outputs_denovo2, "prmsd")

# Dumping output models to PDB
os.makedirs(ODIR, exist_ok=True)
print(f"Writing outputs to {ODIR}")
PLACER.protocol.dump_output(outputs_denovo2, f"{ODIR}/{pl_inp.name()}")

df = pd.DataFrame.from_dict({k: [outputs_denovo2[n][k] for n in outputs_denovo2] for k in outputs_denovo2[0].keys() if k not in ["item", "model", "center"]})


# Generating plots of RMSD vs confidence metrics
plt.figure(figsize=(16, 4))
plt.suptitle(pl_inp.name(), weight="bold", size=14)
for i,k in enumerate(["prmsd", "plddt", "plddt_pde"]):
    plt.subplot(1, 3, i+1)
    plt.scatter(x=df["rmsd"], y=df[k])
    plt.xlabel("lig rmsd", weight="bold", size=14)
    plt.ylabel(k, weight="bold", size=14)
    plt.xticks(size=14)
    plt.yticks(size=14)
plt.tight_layout()
plt.show()

In [ ]:
print("""
##############################################
# Predicting a designed heme-binding protein #
# Creating a bond between heme and HIS       #
##############################################
""")

ODIR = "example_out_API/outputs_denovo_bond"
pdbfile = f"{DIR}/inputs/dnHEM1.pdb"

pl_inp = PLACER.PLACERinput()
pl_inp.pdb(pdbfile)
pl_inp.name(os.path.basename(pdbfile).replace(".pdb", ""))
pl_inp.ligand_reference({"HEM": f"{DIR}/ligands/HEM.mol2"})
pl_inp.bonds([[("A", 149, "HIS", "NE2"), ("B", 213, "HEM", "FE1"), 2.3]])
outputs_denovo_bond = placer.run(pl_inp, 50)

os.makedirs(ODIR, exist_ok=True)
print(f"Writing outputs to {ODIR}")
PLACER.protocol.dump_output(outputs_denovo_bond, f"{ODIR}/{pl_inp.name()}", rerank="prmsd")

df = pd.DataFrame.from_dict({k: [outputs_denovo_bond[n][k] for n in outputs_denovo_bond] for k in outputs_denovo_bond[0].keys() if k not in ["item", "model"]})


In [ ]:
print("""
##############################################
# Predicting a designed heme-binding protein #
# Mutating ARG-44 to TRP at inference        #
#    PDB input generated by Rosetta          #
##############################################
""")

pdbfile = f"{DIR}/inputs/dnHEM1.pdb"
ODIR = "example_out_API/outputs_mutate"

pl_inp = PLACER.PLACERinput()
pl_inp.pdb(pdbfile)
pl_inp.name(os.path.basename(pdbfile).replace(".pdb", ""))
pl_inp.ligand_reference({"HEM": f"{DIR}/ligands/HEM.mol2"})
pl_inp.mutate({("A", 44): "TRP"})
outputs_mutate = placer.run(pl_inp, 10)

os.makedirs(ODIR, exist_ok=True)
print(f"Writing outputs to {ODIR}")
PLACER.protocol.dump_output(outputs_mutate, f"{ODIR}/{pl_inp.name()}", rerank="prmsd")

df = pd.DataFrame.from_dict({k: [outputs_mutate[n][k] for n in outputs_mutate] for k in outputs_mutate[0].keys() if k not in ["item", "model"]})


In [ ]:
print("""
##############################################
# Predicting a designed serine hydrolase     #
# Omitting the ligand during prediction      #
# performing sidechain analysis in apo state #
##############################################
""")

pdbfile = f"{DIR}/inputs/denovo_SER_hydrolase.pdb"
ODIR = "example_out_API/outputs_SER"

pl_inp = PLACER.PLACERinput()
pl_inp.pdb(pdbfile)
pl_inp.exclude_sm(True)
pl_inp.name(os.path.basename(pdbfile).replace(".pdb", ""))
outputs_SER = placer.run(pl_inp, 10)

os.makedirs(ODIR, exist_ok=True)
print(f"Writing outputs to {ODIR}")
PLACER.protocol.dump_output(outputs_SER, f"{ODIR}/{pl_inp.name()}", rerank="prmsd")

df = pd.DataFrame.from_dict({k: [outputs_SER[n][k] for n in outputs_SER] for k in outputs_SER[0].keys() if k not in ["item", "model"]})


In [ ]:
print("""
#####################################################
# Predicting a designed serine hydrolase            #
# Mutating SER-128 to covalently modified SER       #
# Editing the internal residue library at inference #
#####################################################
""")

pdbfile = f"{DIR}/inputs/denovo_SER_hydrolase.pdb"
ODIR = "example_out_API/outputs_mutate_SER"

residue_75I = json.load(open("ligands/75I.json", "r"))

pl_inp = PLACER.PLACERinput()
pl_inp.pdb(pdbfile)
pl_inp.exclude_sm(True)
pl_inp.name(os.path.basename(pdbfile).replace(".pdb", ""))
pl_inp.add_custom_residues( residue_75I )
pl_inp.mutate({("A", 128): "75I"})
outputs_mutate_SER = placer.run(pl_inp, 10)

os.makedirs(ODIR, exist_ok=True)
print(f"Writing outputs to {ODIR}")
PLACER.protocol.dump_output(outputs_mutate_SER, f"{ODIR}/{pl_inp.name()}", rerank="prmsd")

df = pd.DataFrame.from_dict({k: [outputs_mutate_SER[n][k] for n in outputs_mutate_SER] for k in outputs_mutate_SER[0].keys() if k not in ["item", "model"]})


In [ ]:
print("""
################################################
# Predicting a designed heme-binding protein.  #
# Specifying four points as corruption centers #
################################################
""")

pdbfile = f"{DIR}/inputs/dnHEM1.pdb"

ODIR = "example_out_API/outputs_denovo_points"

pl_inp = PLACER.PLACERinput()
pl_inp.pdb(pdbfile)
pl_inp.name(os.path.basename(pdbfile).replace(".pdb", ""))
pl_inp.ligand_reference({"HEM": f"{DIR}/ligands/HEM.mol2"})

# Defining four points as corruption centers
# one point will be randomly selected at each iteration
# This shows how you could sample different points around the protein
# For global docking one could envision generating these points more systematically
pl_inp.corruption_centers( [[5.283,  10.943,   0.990],   # FE1 of heme
                              [12.925,   8.450,  -0.927],  # O1 of heme
                              [-0.240,  12.543,  -5.775],  # OD2 of ASP-40, at the pocket entrance
                              ("A", 166, "TYR", "OH")] ) # OH of TYR-166 on the surface of the protein

outputs_denovo_points = placer.run(pl_inp, 50)

outputs_denovo_points = PLACER.utils.rank_outputs(outputs_denovo_points, "prmsd")

os.makedirs(ODIR, exist_ok=True)
print(f"Writing outputs to {ODIR}")
PLACER.protocol.dump_output(outputs_denovo_points, f"{ODIR}/{pl_inp.name()}")

df = pd.DataFrame.from_dict({k: [outputs_denovo_points[n][k] for n in outputs_denovo_points] for k in outputs_denovo_points[0].keys() if k not in ["item", "model"]})
